# Admissions Classifier

Binary classifier that predicts **Accepted** vs **Rejected** for applications using GPA, SAT score, and extracurricular activities.

**Objective**: Train a neural network on the [Kaggle student admission dataset](https://www.kaggle.com/datasets/amanace/student-admission-dataset).

**Model type**: Binary classification (sigmoid output, binary cross-entropy).

**Workflow**:
1. Load data
2. Preprocess and binarize target (Accepted=1, Rejected/Waitlisted=0)
3. Train/test split, scale features
4. Build and train neural network
5. Evaluate and save model + scaler

## Run in the browser (no local setup)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adzuci/admissions-classifier/blob/main/admissions_classifier.ipynb)

Works on **Colab**, **Windows**, and **Mac**. In Colab: upload `student_admission_dataset.csv` when prompted (download from [Kaggle](https://www.kaggle.com/datasets/amanace/student-admission-dataset)).

In [1]:
try:
    import google.colab
    get_ipython().run_line_magic('pip', 'install -q tensorflow pandas joblib')
except (ImportError, NameError):
    pass  # local: deps assumed installed

## STEP 1: Imports and SimpleScaler

In [2]:
import os
import sys

# Mac: Metal/GPU can hang; force CPU. Colab & Windows: use defaults (GPU if available).
if sys.platform == "darwin":
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import joblib
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers


class SimpleScaler:
    """Minimal scaler with fit/transform (no sklearn)."""

    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        self.scale_[self.scale_ == 0] = 1.0
        return self

    def transform(self, X):
        return (X - self.mean_) / self.scale_


print(f"TensorFlow: {keras.__version__}")

TensorFlow: 3.12.1


## STEP 2: Load data

In [3]:
# Colab: upload CSV when prompted. Local (Mac/Windows): use student_admission_dataset.csv
try:
    from google.colab import files
    uploaded = files.upload()
    csv_name = list(uploaded.keys())[0]
except ImportError:
    csv_name = "student_admission_dataset.csv"
# Download from Kaggle if missing: kaggle datasets download -d amanace/student-admission-dataset

df = pd.read_csv(csv_name)
df.head(10)

,GPA,SAT_Score,Extracurricular_Activities,Admission_Status
0,3.46,1223,8,Rejected
1,2.54,974,8,Rejected
2,2.91,909,9,Rejected
3,2.83,1369,5,Accepted
4,3.60,1536,7,Accepted
5,3.52,1476,9,Accepted
6,3.84,1002,8,Rejected
7,2.63,975,6,Waitlisted
8,3.13,1450,8,Waitlisted
9,2.54,1118,7,Rejected


## STEP 3: Preprocess

In [4]:
# Binarize: Accepted=1, Rejected/Waitlisted=0
df["Admit"] = (df["Admission_Status"] == "Accepted").astype(int)

feature_cols = ["GPA", "SAT_Score", "Extracurricular_Activities"]
X = df[feature_cols].values.astype(np.float32)
y = df["Admit"].values

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Class balance: Admit=1 {y.sum()}, Admit=0 {len(y) - y.sum()}")

X shape: (250, 3), y shape: (250,)
Class balance: Admit=1 81, Admit=0 169


In [5]:
# Train/test split (80/20)
np.random.seed(42)
idx = np.random.permutation(len(X))
split = int(0.8 * len(X))
train_idx, test_idx = idx[:split], idx[split:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# Scale features
scaler = SimpleScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

## STEP 4: Build and train model

In [6]:
model = keras.Sequential([
    keras.Input(shape=(3,)),
    layers.Dense(8, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 41 (164.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train_s, y_train, epochs=20, batch_size=32, verbose=1)

Epoch 1/20


## STEP 5: Evaluate

In [ ]:
loss, acc = model.evaluate(X_test_s, y_test, verbose=0)
print(f"Test accuracy: {acc:.4f}")

preds = (model.predict(X_test_s, verbose=0) > 0.5).astype(int).flatten()
print(f"Sample predictions: {preds[:15]}...")

## STEP 6: Save model and scaler

In [ ]:
model.save("model.keras")
joblib.dump(scaler, "scaler.joblib")
print("Saved model.keras and scaler.joblib")
print("In Colab: use Files pane to download these files.")